### 1.direction of car and coordinate

In [1]:
#!/usr/bin/env python
# coding: utf-8
# # this is a heading 
# In[49]:
import copy
import numpy as np
from enum import Enum
import bisect 
import time

class Coordinate():
  def __init__(self, X, Y):
    self.X = X 
    self.Y = Y
  def __str__(self):
    return "(" + str(self.X) + ", " + str(self.Y) + ")"

class Direction(Enum):
  HORIZONTAL = 1
  VERTICAL = 2


### 2.car class

In [2]:
class Car(object):
  def __init__(self, carLetter, coordinate, fuel):
    self.carLetter = carLetter 
    self.coordinates = []
    self.coordinates.append(coordinate)
    self.fuel = int(fuel)
    self.path=[];

  def isA(self):
    return self.carLetter == 'A'

  def move(self, amount):
    if self.direction == Direction.HORIZONTAL:
      for c in self.coordinates:
        c.Y += amount
        
    else:
      for c in self.coordinates:
        c.X += amount
        
    self.fuel -= abs(amount)
    self.path.append(amount)
    
  def canMove(self):
    return self.fuel > 0

  def compareLetters(self, carLetter):
    return self.carLetter == carLetter

  def addCoordinate(self, newCoordinate):
    self.coordinates.append(newCoordinate)
    # Update direction
    # L-R (Horizontal)
    if self.coordinates[0].X == newCoordinate.X and self.coordinates[0].Y != newCoordinate.Y:
      self.direction = Direction.HORIZONTAL
    # U-D (Vertical)
    else:
      self.direction = Direction.VERTICAL

  def getLetter(self):
    return self.carLetter; 

  # Primarily for testing to see if Car works
  def __str__(self):
    s = self.carLetter + " ------ ";
    for i in self.coordinates:
      s += str(i)
    return s + " | " + str(self.fuel) + " | " + str(self.direction)

  def create(carLetter1, coordinates, fuel):
    return Car(carLetter1, coordinates, fuel)



### 3. 6X6 car board class

In [3]:
class Board(object):
  def __init__(self, cars):
    self.grid = [['.' for i in range(6)] for j in range(6)]
    self.size = {'X': 6, 'Y': 6}
    self.cars = []
    for car in cars:  
      c = car
      self.cars.append(copy.deepcopy(c));
      for coord in c.coordinates:
        self.grid[coord.X][coord.Y] = car.carLetter
        
  def __eq__(self, obj):
        return self.grid == obj.grid

  def findCarPaths(self, car):
    if car.canMove():
      fwd = self.checkPathForward(car)
      bwd = self.checkPathBackward(car)
      carPath = bwd + fwd
    else:
      carPath = []
    return carPath
  
  def moveCar(self, carLetter1, amount):
    for car in self.cars: 
      if car.carLetter == carLetter1:
        for coord in car.coordinates:
          #print(coord)
          self.grid[coord.X][coord.Y] = "."
        car.move(amount)
        for coord in car.coordinates:
          #print(coord)
          self.grid[coord.X][coord.Y] = carLetter1

        # Is this a regular car that just left?
        if car.carLetter == self.grid[2][5] and car.direction == Direction.HORIZONTAL and car.carLetter != "A":
          for coord in car.coordinates:
            self.grid[coord.X][coord.Y] = "."
          self.cars.remove(car)  
         

  def checkPathForward(self, car):
    dir = car.direction
    lastCoord = car.coordinates[len(car.coordinates)-1]
    if dir == Direction.HORIZONTAL:
      startPoint = lastCoord.Y
    else:
      startPoint = lastCoord.X

    possibleMoves = []
    for i in range(startPoint+1, 6):
        #fuel req
      if(car.fuel < i-startPoint):
        continue;
      # Find next position to check    
      if dir == Direction.HORIZONTAL:
        gridPos = self.grid[lastCoord.X][i] 
      else:
        gridPos = self.grid[i][lastCoord.Y]

      # If there's an open position, add and continue
      # Else, there is something blocking the way and there's no way to continue any more
      if gridPos == ".":
        possibleMoves.append(i-startPoint)
      else:
        break;

    return possibleMoves

 
  def checkPathBackward(self, car):
    dir = car.direction
    lastCoord = car.coordinates[0]
    if dir == Direction.HORIZONTAL:
      startPoint = lastCoord.Y
    else:
      startPoint = lastCoord.X

    possibleMoves = []
    for i in range(startPoint, 0, -1):
      if (car.fuel <= (startPoint-i)):
        continue
      # Find next position to check
      if dir == Direction.HORIZONTAL:
        gridPos = self.grid[lastCoord.X][i-1] 
      else:
        gridPos = self.grid[i-1][lastCoord.Y]

      # If there's an open position, add and continue
      # Else, there is something blocking the way and there's no way to continue any more
      if gridPos == ".":
        possibleMoves.append(i-startPoint-1)
      else:
        break;

    return possibleMoves

  def winConditionMet(self):
    if self.grid[2][5] == "A":
      return True      
    return False

  def board_h1_value(self):
    for value in range(6):
        if self.grid[2][value]=="A":
            if self.grid[2][value+1]=="A":
                return 5-(value+1)
            return 5-value

  def __str__(self): 
    s = ""
    for x in range(6):
      for y in range(6):
        s += self.grid[x][y]
      s += "\n"
      
    return s


  #Heuristic definitions:

  #h1 = The number of blocking vehicles
  def h1(self):
        count = 0
        last = " "
        for i in range(6):
            e = self.grid[2][5-i]
            if e == "A":
                break
            elif e == last:
                continue
            elif e != ".":
                count += 1
                last = e
        return count
  
  #h2 = The number of blocked positions
  def h2(self):
        count = 0
        for i in range(6):
            e = self.grid[2][5-i]
            if e == "A":
                break
            elif e != ".":
                count += 1
        return count
    
  #h3 = The number of blocking vehicles (h1) multiplied by a constant
  def h3(self, constant):
        return self.h1()*constant
    
  #h4 = Is there a car blocking the exit square (3f)
  #Yes: h4 = 1
  #No:  h4 = 0

  def h4(self):
        if self.grid[2][5] != "." and self.grid[2][5] != "A":
            return 1
        else:
            return 0



### 4.board information display

In [4]:
demos = []
demoFuels = []

with open('sample-input.txt') as f:
    for line in f:
      if not line.startswith(('#', '\n')):##chack the starting if '#'
        in_arr = np.array([line.strip()])
        data = np.char.split(in_arr)[0]
        demos.append(data)
f.close()

print(demos)


# # Creating boards

# In[64]:


boards = []
for data in demos:
  # Seperate Grid Data from the Fuel Parameters
  grid = data[0];
  fuels = []
  if len(data) > 1:
    fuels.append(data[1:])

  cars = []
  for row in range(6):
    for col in range(6):
      current = grid[row*6 + col];
      
      # If Unique: Create a car and append
      # If Not Unique: Concat to same letter
      if current == ".":
        continue

      unique = True
      for c in cars:
        if c.compareLetters(current):
          c.addCoordinate(Coordinate(row,col))
          unique = False

      if unique:
        # Now check if there's a unique fuel amount or if its the default 100
        fuelAmount = 100
        if len(fuels) > 0:
          for f in fuels[0]:
            if current == f[0]:
              fuelAmount = f[1:]

        cars.append(Car.create(current, Coordinate(row,col), fuelAmount));

  # Checking if all cars were made properly
  print("CARS: ")
  print("--------")
  for c in cars:
    print(c)
  print()
  
  board = Board(cars)
  # Checking if all boards were made properly
  print("BOARD: ")
  print("--------")
  print(board)
  print()

  boards.append(board)


  for b in boards:
    for c in b.cars:
      print(c)
    print()



[['BBIJ....IJCC..IAAMGDDK.MGH.KL.GHFFL.'], ['..I...BBI.K.GHAAKLGHDDKLG..JEEFF.J..'], ['JBBCCCJDD..MJAAL.MFFKL.N..KGGN.HH...'], ['BBB..MCCDD.MAAKL.MJ.KLEEJ.GG..JHHHII', 'J0', 'B4'], ['IJBBCCIJDDL.IJAAL.EEK.L...KFF..GGHH.', 'F0', 'G6'], ['BB.G.HE..G.HEAAG.I..FCCIDDF..I..F...']]
CARS: 
--------
B ------ (0, 0)(0, 1) | 100 | Direction.HORIZONTAL
I ------ (0, 2)(1, 2)(2, 2) | 100 | Direction.VERTICAL
J ------ (0, 3)(1, 3) | 100 | Direction.VERTICAL
C ------ (1, 4)(1, 5) | 100 | Direction.HORIZONTAL
A ------ (2, 3)(2, 4) | 100 | Direction.HORIZONTAL
M ------ (2, 5)(3, 5) | 100 | Direction.VERTICAL
G ------ (3, 0)(4, 0)(5, 0) | 100 | Direction.VERTICAL
D ------ (3, 1)(3, 2) | 100 | Direction.HORIZONTAL
K ------ (3, 3)(4, 3) | 100 | Direction.VERTICAL
H ------ (4, 1)(5, 1) | 100 | Direction.VERTICAL
L ------ (4, 4)(5, 4) | 100 | Direction.VERTICAL
F ------ (5, 2)(5, 3) | 100 | Direction.HORIZONTAL

BOARD: 
--------
BBIJ..
..IJCC
..IAAM
GDDK.M
GH.KL.
GHFFL.


B ------ (0, 0)(0, 1) | 100 | Direc

### 5.uniform cast search for example input

In [5]:
import sys

    
class Node(object):
  def __init__(self, board, h1,gn):
    self.board = board
    self.h=0+h1
    self.gn=0+gn;
    self.path = []
    self.direction='none'
    self.parent = 0
  def __lt__(self, other):
        return self.h+self.gn < other.h+other.gn
  def __eq__(self, other):
    if isinstance(other,Node):
        return self.board == other.board
    
class UCS(object):
  def __init__(self):
    self.opened = []
    self.closed = []
    self.steps=-1;
    self.path=[]
  def printnode(self,node):
    print(node.board)
    print("f(n):"+str(node.gn+node.h),end='')
    print(" g(n):"+str(node.gn),end='')
    print(" h(n):"+str(node.h),end=' ')
    
    for car in node.board.cars:
        if car.fuel<100:
            print(car.carLetter+": fuel:"+str(car.fuel), end = ' ')
            for i in range(len(car.path)):
                if car.direction==Direction.HORIZONTAL:
                    if(car.path[i]>0):
                        print("right:"+str(abs(car.path[i])), end = ' ')

                    if(car.path[i]<0):
                        print("left:"+str(abs(car.path[i])), end = ' ')


                if car.direction==Direction.VERTICAL:
                    if(car.path[i]>0):
                        print("Down:"+str(abs(car.path[i])), end = ' ')

                    if(car.path[i]<0):
                        print("UP:"+str(abs(car.path[i])), end = ' ')
               
    print("\n\n")
    
  def recursivePrinting(self, node):
    print(node.board)
    print("g(n):"+str(node.gn),end='')
    print(" h(n):"+str(node.h),end='')
    print(" f(n):"+str(node.gn+node.h))
    for car in node.board.cars:
        if car.fuel<100:
            print(car.carLetter+": fuel:"+str(car.fuel), end = ' ')
            for i in range(len(car.path)):
                if car.direction==Direction.HORIZONTAL:
                    if(car.path[i]>0):
                        print("right:"+str(abs(car.path[i])), end = ' ')

                    if(car.path[i]<0):
                        print("left:"+str(abs(car.path[i])), end = ' ')


                if car.direction==Direction.VERTICAL:
                    if(car.path[i]>0):
                        print("Down:"+str(abs(car.path[i])), end = ' ')

                    if(car.path[i]<0):
                        print("UP:"+str(abs(car.path[i])), end = ' ')
            print("")    
    print("\n\n")        
        
    
    if node.parent==0:
        return
    else:
      self.recursivePrinting(node.parent)
      
  def findsteps(self,node):
    self.steps+=1;
    self.path.append(node.path)
    if node.parent==0:
        return
    else:
      self.findsteps(node.parent)
    
  def findgn(self,node):
    
    if node.parent==0:
        return
    else:
      node.gn+=node.parent.h
      self.findgn(node.parent)
    
  def search(self, board,output_solution_file,output_search_file):
    original_stdout = sys.stdout # Save a reference to the original standard output

    with open(output_solution_file, 'w') as f, open(output_search_file, 'w') as fb:
        sys.stdout = f
        
        print("Initial board configuration")
        print()
        print(board)
        print("Car Fuel Available: ", end = '')
        for i in range(len(board.cars)):
            print(""+board.cars[i].carLetter+":"+str(board.cars[i].fuel)+", ", end ='')
        print()
        print("---------------------------running:---------------------------")
        #Initialise first Node with cost 0 (different for other algorithms)
        a = Node(copy.deepcopy(board), 0,0)
        self.opened.append(a)

        start_time = time.time()
        statessearched = 0
        sys.stdout = fb
        while(1):              
            #If open list is empty, exit
            
            if(len(self.opened)==0):
                
                print("No Solution")
                sys.stdout = f
                print("No Solution")
                sys.stdout = original_stdout
                return

            #Pop the best candidate node
            curr = self.opened.pop(0)

            #If the popped node is the solution, exit out and print stuff (later to file)



            #print("CHECKING CLOSED + OPEN: ")
            #print("-----")
            #for c in self.closed:
            #  print(c.board)
            #print("-----")

            #for c in self.opened:
            #  print(c.board)
            #print("----- END")

            #If it's not a solution insert into closed list and increment counter
            self.closed.append(curr)

          #
            #print()
            for c in curr.board.cars:

                paths = curr.board.findCarPaths(c)    
                if len(paths) > 0:  
                    #print("Car ", c.carLetter, " can move:", paths)

                    for i in range(len(paths)):
                        #Create a new node for successor
                        #Increase the cost of this new node (different in other algorithms)
                        newnode=0
                        if curr.parent!=0:
                            newnode = Node(Board(curr.board.cars), curr.h+1,0)
                        else:
                            newnode = Node(Board(curr.board.cars), curr.h+1,0)

                        newnode.parent = curr                 
                        if(newnode.board.winConditionMet()):
                            sys.stdout = f
                            statessearched+=1



                            print("Runtime: %s seconds" % (time.time() - start_time))
                            self.findsteps(curr)
                            print("Number of moves: ", self.steps)
                            print("Search path length: %s states" % statessearched)
                            for node in self.path:
                                for path in node:
                                    print(""+path+", ", end ='')
                            print()

                            print("------")
                            print("---------------------------solution:---------------------------")
                            self.recursivePrinting(curr)
                            print("end")
                            sys.stdout = original_stdout
                            return

                                    #Apply the move
                        newnode.board.moveCar(c.carLetter, paths[i])
                        newnode.path.append(c.carLetter+": move:"+str(paths[i]))
                       

                        for node in self.opened:
                            if newnode.gn+newnode.h>node.gn+node.h:
                              #  print(str(newnode.h)+">"+str(node.h))
                                continue

                        if newnode in self.opened or newnode in self.closed:
                            continue

                        #Insert into open list (in sorted order)
                        self.printnode(newnode)
                        statessearched+=1
                        newnode.gn+=1
                        bisect.insort(self.opened, newnode)
                        
                        
                                

currentBoard = Board(boards[0].cars)
ucs = UCS()
ucs.search(currentBoard,'ucs_solution_1.txt','usc_search_1.txt')

currentBoard = Board(boards[1].cars)
ucs = UCS()
ucs.search(currentBoard,'ucs_solution_2.txt','usc_search_2.txt')
currentBoard = Board(boards[2].cars)
ucs = UCS()
ucs.search(currentBoard,'ucs_solution_3.txt','usc_search_3.txt')
currentBoard = Board(boards[3].cars)
ucs = UCS()
ucs.search(currentBoard,'ucs_solution_4.txt','usc_search_4.txt')
currentBoard = Board(boards[4].cars)
ucs = UCS()
ucs.search(currentBoard,'ucs_solution_5.txt','usc_search_5.txt')
currentBoard = Board(boards[5].cars)
ucs = UCS()
ucs.search(currentBoard,'ucs_solution_6.txt','usc_search_6.txt')

### Greedy Best First Search h1

In [7]:
import sys

    
class Node(object):
  def __init__(self, board, h1,gn):
    self.board = board
    self.h=0+h1
    self.gn=0+gn;
    self.path = []
    self.direction='none'
    self.parent = 0
  def __lt__(self, other):
        return self.h+self.gn < other.h+other.gn
  def __eq__(self, other):
    if isinstance(other,Node):
        return self.board == other.board
    
class GBFS(object):
  def __init__(self):
    self.opened = []
    self.closed = []
    self.steps=-1;
    self.path=[]
  def printnode(self,node):
    print(node.board)
    print("f(n):"+str(node.gn+node.h),end='')
    print(" g(n):"+str(node.gn),end='')
    print(" h(n):"+str(node.h),end=' ')
    
    for car in node.board.cars:
        if car.fuel<100:
            print(car.carLetter+": fuel:"+str(car.fuel), end = ' ')
            for i in range(len(car.path)):
                if car.direction==Direction.HORIZONTAL:
                    if(car.path[i]>0):
                        print("right:"+str(abs(car.path[i])), end = ' ')

                    if(car.path[i]<0):
                        print("left:"+str(abs(car.path[i])), end = ' ')


                if car.direction==Direction.VERTICAL:
                    if(car.path[i]>0):
                        print("Down:"+str(abs(car.path[i])), end = ' ')

                    if(car.path[i]<0):
                        print("UP:"+str(abs(car.path[i])), end = ' ')
               
    print("\n\n")
    
  def recursivePrinting(self, node):
    print(node.board)
    print("g(n):"+str(node.gn),end='')
    print(" h(n):"+str(node.h),end='')
    print(" f(n):"+str(node.gn+node.h))
    for car in node.board.cars:
        if car.fuel<100:
            print(car.carLetter+": fuel:"+str(car.fuel), end = ' ')
            for i in range(len(car.path)):
                if car.direction==Direction.HORIZONTAL:
                    if(car.path[i]>0):
                        print("right:"+str(abs(car.path[i])), end = ' ')

                    if(car.path[i]<0):
                        print("left:"+str(abs(car.path[i])), end = ' ')


                if car.direction==Direction.VERTICAL:
                    if(car.path[i]>0):
                        print("Down:"+str(abs(car.path[i])), end = ' ')

                    if(car.path[i]<0):
                        print("UP:"+str(abs(car.path[i])), end = ' ')
            print("")    
    print("\n\n")        
        
    
    if node.parent==0:
        return
    else:
      self.recursivePrinting(node.parent)
      
  def findsteps(self,node):
    self.steps+=1;
    self.path.append(node.path)
    if node.parent==0:
        return
    else:
      self.findsteps(node.parent)
    
  def findgn(self,node):
    
    if node.parent==0:
        return
    else:
      node.gn+=node.parent.h
      self.findgn(node.parent)
    
  def search(self, board,output_solution_file,output_search_file):
    original_stdout = sys.stdout # Save a reference to the original standard output

    with open(output_solution_file, 'w') as f, open(output_search_file, 'w') as fb:
        sys.stdout = f
        
        print("Initial board configuration")
        print()
        print(board)
        print("Car Fuel Available: ", end = '')
        for i in range(len(board.cars)):
            print(""+board.cars[i].carLetter+":"+str(board.cars[i].fuel)+", ", end ='')
        print()
        print("---------------------------running:---------------------------")
        #Initialise first Node with cost 0 (different for other algorithms)
        a = Node(copy.deepcopy(board), 0,0)
        self.opened.append(a)

        start_time = time.time()
        statessearched = 0
        sys.stdout = fb
        while(1):              
            #If open list is empty, exit
            
            if(len(self.opened)==0):
                
                print("No Solution")
                sys.stdout = f
                print("No Solution")
                sys.stdout = original_stdout
                return

            #Pop the best candidate node
            curr = self.opened.pop(0)

            #If the popped node is the solution, exit out and print stuff (later to file)



            #print("CHECKING CLOSED + OPEN: ")
            #print("-----")
            #for c in self.closed:
            #  print(c.board)
            #print("-----")

            #for c in self.opened:
            #  print(c.board)
            #print("----- END")

            #If it's not a solution insert into closed list and increment counter
            self.closed.append(curr)

          #
            #print()
            for c in curr.board.cars:

                paths = curr.board.findCarPaths(c)    
                if len(paths) > 0:  
                    #print("Car ", c.carLetter, " can move:", paths)

                    for i in range(len(paths)):
                        #Create a new node for successor
                        #Increase the cost of this new node (different in other algorithms)
                        newnode=0
                        if curr.parent!=0:
                            newnode = Node(Board(curr.board.cars), curr.board.h1(),0)
                        else:
                            newnode = Node(Board(curr.board.cars), curr.board.h1(),0)

                        newnode.parent = curr                 
                        if(newnode.board.winConditionMet()):
                            sys.stdout = f
                            statessearched+=1



                            print("Runtime: %s seconds" % (time.time() - start_time))
                            self.findsteps(curr)
                            print("Number of moves: ", self.steps)
                            print("Search path length: %s states" % statessearched)
                            for node in self.path:
                                for path in node:
                                    print(""+path+", ", end ='')
                            print()

                            print("------")
                            print("---------------------------solution:---------------------------")
                            self.recursivePrinting(curr)
                            print("end")
                            sys.stdout = original_stdout
                            return

                                    #Apply the move
                        newnode.board.moveCar(c.carLetter, paths[i])
                        newnode.path.append(c.carLetter+": move:"+str(paths[i]))
                       

                        for node in self.opened:
                            if newnode.gn+newnode.h>node.gn+node.h:
                              #  print(str(newnode.h)+">"+str(node.h))
                                continue

                        if newnode in self.opened or newnode in self.closed:
                            continue

                        #Insert into open list (in sorted order)
                        self.printnode(newnode)
                        statessearched+=1
                        newnode.gn+=1
                        bisect.insort(self.opened, newnode)
                        
currentBoard = Board(boards[0].cars)
ucs = GBFS()
ucs.search(currentBoard,'GBFS_solution_h1_1.txt','HBFS_search_h1_1.txt')                        

currentBoard = Board(boards[1].cars)
ucs = GBFS()
ucs.search(currentBoard,'GBFS_solution_h1_2.txt','HBFS_search_h1_2.txt')
currentBoard = Board(boards[2].cars)
ucs = GBFS()
ucs.search(currentBoard,'GBFS_solution_h1_3.txt','HBFS_search_h1_3.txt')
currentBoard = Board(boards[3].cars)
ucs = GBFS()
ucs.search(currentBoard,'GBFS_solution_h1_4.txt','HBFS_search_h1_4.txt')
currentBoard = Board(boards[4].cars)
ucs = GBFS()
ucs.search(currentBoard,'GBFS_solution_h1_5.txt','HBFS_search_h1_5.txt')
currentBoard = Board(boards[5].cars)
ucs = GBFS()
ucs.search(currentBoard,'GBFS_solution_h1_6.txt','HBFS_search_h1_6.txt')

### Greedy Best First Search h2

In [8]:
import sys

    
class Node(object):
  def __init__(self, board, h1,gn):
    self.board = board
    self.h=0+h1
    self.gn=0+gn;
    self.path = []
    self.direction='none'
    self.parent = 0
  def __lt__(self, other):
        return self.h+self.gn < other.h+other.gn
  def __eq__(self, other):
    if isinstance(other,Node):
        return self.board == other.board
    
class GBFS(object):
  def __init__(self):
    self.opened = []
    self.closed = []
    self.steps=-1;
    self.path=[]
  def printnode(self,node):
    print(node.board)
    print("f(n):"+str(node.gn+node.h),end='')
    print(" g(n):"+str(node.gn),end='')
    print(" h(n):"+str(node.h),end=' ')
    
    for car in node.board.cars:
        if car.fuel<100:
            print(car.carLetter+": fuel:"+str(car.fuel), end = ' ')
            for i in range(len(car.path)):
                if car.direction==Direction.HORIZONTAL:
                    if(car.path[i]>0):
                        print("right:"+str(abs(car.path[i])), end = ' ')

                    if(car.path[i]<0):
                        print("left:"+str(abs(car.path[i])), end = ' ')


                if car.direction==Direction.VERTICAL:
                    if(car.path[i]>0):
                        print("Down:"+str(abs(car.path[i])), end = ' ')

                    if(car.path[i]<0):
                        print("UP:"+str(abs(car.path[i])), end = ' ')
               
    print("\n\n")
    
  def recursivePrinting(self, node):
    print(node.board)
    print("g(n):"+str(node.gn),end='')
    print(" h(n):"+str(node.h),end='')
    print(" f(n):"+str(node.gn+node.h))
    for car in node.board.cars:
        if car.fuel<100:
            print(car.carLetter+": fuel:"+str(car.fuel), end = ' ')
            for i in range(len(car.path)):
                if car.direction==Direction.HORIZONTAL:
                    if(car.path[i]>0):
                        print("right:"+str(abs(car.path[i])), end = ' ')

                    if(car.path[i]<0):
                        print("left:"+str(abs(car.path[i])), end = ' ')


                if car.direction==Direction.VERTICAL:
                    if(car.path[i]>0):
                        print("Down:"+str(abs(car.path[i])), end = ' ')

                    if(car.path[i]<0):
                        print("UP:"+str(abs(car.path[i])), end = ' ')
            print("")    
    print("\n\n")        
        
    
    if node.parent==0:
        return
    else:
      self.recursivePrinting(node.parent)
      
  def findsteps(self,node):
    self.steps+=1;
    self.path.append(node.path)
    if node.parent==0:
        return
    else:
      self.findsteps(node.parent)
    
  def findgn(self,node):
    
    if node.parent==0:
        return
    else:
      node.gn+=node.parent.h
      self.findgn(node.parent)
    
  def search(self, board,output_solution_file,output_search_file):
    original_stdout = sys.stdout # Save a reference to the original standard output

    with open(output_solution_file, 'w') as f, open(output_search_file, 'w') as fb:
        sys.stdout = f
        
        print("Initial board configuration")
        print()
        print(board)
        print("Car Fuel Available: ", end = '')
        for i in range(len(board.cars)):
            print(""+board.cars[i].carLetter+":"+str(board.cars[i].fuel)+", ", end ='')
        print()
        print("---------------------------running:---------------------------")
        #Initialise first Node with cost 0 (different for other algorithms)
        a = Node(copy.deepcopy(board), 0,0)
        self.opened.append(a)

        start_time = time.time()
        statessearched = 0
        sys.stdout = fb
        while(1):              
            #If open list is empty, exit
            
            if(len(self.opened)==0):
                
                print("No Solution")
                sys.stdout = f
                print("No Solution")
                sys.stdout = original_stdout
                return

            #Pop the best candidate node
            curr = self.opened.pop(0)

            #If the popped node is the solution, exit out and print stuff (later to file)



            #print("CHECKING CLOSED + OPEN: ")
            #print("-----")
            #for c in self.closed:
            #  print(c.board)
            #print("-----")

            #for c in self.opened:
            #  print(c.board)
            #print("----- END")

            #If it's not a solution insert into closed list and increment counter
            self.closed.append(curr)

          #
            #print()
            for c in curr.board.cars:

                paths = curr.board.findCarPaths(c)    
                if len(paths) > 0:  
                    #print("Car ", c.carLetter, " can move:", paths)

                    for i in range(len(paths)):
                        #Create a new node for successor
                        #Increase the cost of this new node (different in other algorithms)
                        newnode=0
                        if curr.parent!=0:
                            newnode = Node(Board(curr.board.cars), curr.board.h2(),0)
                        else:
                            newnode = Node(Board(curr.board.cars), curr.board.h2(),0)

                        newnode.parent = curr                 
                        if(newnode.board.winConditionMet()):
                            sys.stdout = f
                            statessearched+=1



                            print("Runtime: %s seconds" % (time.time() - start_time))
                            self.findsteps(curr)
                            print("Number of moves: ", self.steps)
                            print("Search path length: %s states" % statessearched)
                            for node in self.path:
                                for path in node:
                                    print(""+path+", ", end ='')
                            print()

                            print("------")
                            print("---------------------------solution:---------------------------")
                            self.recursivePrinting(curr)
                            print("end")
                            sys.stdout = original_stdout
                            return

                                    #Apply the move
                        newnode.board.moveCar(c.carLetter, paths[i])
                        newnode.path.append(c.carLetter+": move:"+str(paths[i]))
                       

                        for node in self.opened:
                            if newnode.gn+newnode.h>node.gn+node.h:
                              #  print(str(newnode.h)+">"+str(node.h))
                                continue

                        if newnode in self.opened or newnode in self.closed:
                            continue

                        #Insert into open list (in sorted order)
                        self.printnode(newnode)
                        statessearched+=1
                        newnode.gn+=1
                        bisect.insort(self.opened, newnode)
                        
currentBoard = Board(boards[0].cars)
ucs = GBFS()
ucs.search(currentBoard,'GBFS_solution_h2_1.txt','HBFS_search_h2_1.txt')                        

currentBoard = Board(boards[1].cars)
ucs = GBFS()
ucs.search(currentBoard,'GBFS_solution_h2_2.txt','HBFS_search_h2_2.txt')
currentBoard = Board(boards[2].cars)
ucs = GBFS()
ucs.search(currentBoard,'GBFS_solution_h2_3.txt','HBFS_search_h2_3.txt')
currentBoard = Board(boards[3].cars)
ucs = GBFS()
ucs.search(currentBoard,'GBFS_solution_h2_4.txt','HBFS_search_h2_4.txt')
currentBoard = Board(boards[4].cars)
ucs = GBFS()
ucs.search(currentBoard,'GBFS_solution_h2_5.txt','HBFS_search_h2_5.txt')
currentBoard = Board(boards[5].cars)
ucs = GBFS()
ucs.search(currentBoard,'GBFS_solution_h2_6.txt','HBFS_search_h2_6.txt')

### Greedy Best First Search h3

In [ ]:
import sys

    
class Node(object):
  def __init__(self, board, h1,gn):
    self.board = board
    self.h=0+h1
    self.gn=0+gn;
    self.path = []
    self.direction='none'
    self.parent = 0
  def __lt__(self, other):
        return self.h+self.gn < other.h+other.gn
  def __eq__(self, other):
    if isinstance(other,Node):
        return self.board == other.board
    
class GBFS(object):
  def __init__(self):
    self.opened = []
    self.closed = []
    self.steps=-1;
    self.path=[]
  def printnode(self,node):
    print(node.board)
    print("f(n):"+str(node.gn+node.h),end='')
    print(" g(n):"+str(node.gn),end='')
    print(" h(n):"+str(node.h),end=' ')
    
    for car in node.board.cars:
        if car.fuel<100:
            print(car.carLetter+": fuel:"+str(car.fuel), end = ' ')
            for i in range(len(car.path)):
                if car.direction==Direction.HORIZONTAL:
                    if(car.path[i]>0):
                        print("right:"+str(abs(car.path[i])), end = ' ')

                    if(car.path[i]<0):
                        print("left:"+str(abs(car.path[i])), end = ' ')


                if car.direction==Direction.VERTICAL:
                    if(car.path[i]>0):
                        print("Down:"+str(abs(car.path[i])), end = ' ')

                    if(car.path[i]<0):
                        print("UP:"+str(abs(car.path[i])), end = ' ')
               
    print("\n\n")
    
  def recursivePrinting(self, node):
    print(node.board)
    print("g(n):"+str(node.gn),end='')
    print(" h(n):"+str(node.h),end='')
    print(" f(n):"+str(node.gn+node.h))
    for car in node.board.cars:
        if car.fuel<100:
            print(car.carLetter+": fuel:"+str(car.fuel), end = ' ')
            for i in range(len(car.path)):
                if car.direction==Direction.HORIZONTAL:
                    if(car.path[i]>0):
                        print("right:"+str(abs(car.path[i])), end = ' ')

                    if(car.path[i]<0):
                        print("left:"+str(abs(car.path[i])), end = ' ')


                if car.direction==Direction.VERTICAL:
                    if(car.path[i]>0):
                        print("Down:"+str(abs(car.path[i])), end = ' ')

                    if(car.path[i]<0):
                        print("UP:"+str(abs(car.path[i])), end = ' ')
            print("")    
    print("\n\n")        
        
    
    if node.parent==0:
        return
    else:
      self.recursivePrinting(node.parent)
      
  def findsteps(self,node):
    self.steps+=1;
    self.path.append(node.path)
    if node.parent==0:
        return
    else:
      self.findsteps(node.parent)
    
  def findgn(self,node):
    
    if node.parent==0:
        return
    else:
      node.gn+=node.parent.h
      self.findgn(node.parent)
    
  def search(self, board,output_solution_file,output_search_file):
    original_stdout = sys.stdout # Save a reference to the original standard output

    with open(output_solution_file, 'w') as f, open(output_search_file, 'w') as fb:
        sys.stdout = f
        
        print("Initial board configuration")
        print()
        print(board)
        print("Car Fuel Available: ", end = '')
        for i in range(len(board.cars)):
            print(""+board.cars[i].carLetter+":"+str(board.cars[i].fuel)+", ", end ='')
        print()
        print("---------------------------running:---------------------------")
        #Initialise first Node with cost 0 (different for other algorithms)
        a = Node(copy.deepcopy(board), 0,0)
        self.opened.append(a)

        start_time = time.time()
        statessearched = 0
        sys.stdout = fb
        while(1):              
            #If open list is empty, exit
            
            if(len(self.opened)==0):
                
                print("No Solution")
                sys.stdout = f
                print("No Solution")
                sys.stdout = original_stdout
                return

            #Pop the best candidate node
            curr = self.opened.pop(0)

            #If the popped node is the solution, exit out and print stuff (later to file)



            #print("CHECKING CLOSED + OPEN: ")
            #print("-----")
            #for c in self.closed:
            #  print(c.board)
            #print("-----")

            #for c in self.opened:
            #  print(c.board)
            #print("----- END")

            #If it's not a solution insert into closed list and increment counter
            self.closed.append(curr)

          #
            #print()
            for c in curr.board.cars:

                paths = curr.board.findCarPaths(c)    
                if len(paths) > 0:  
                    #print("Car ", c.carLetter, " can move:", paths)

                    for i in range(len(paths)):
                        #Create a new node for successor
                        #Increase the cost of this new node (different in other algorithms)
                        newnode=0
                        if curr.parent!=0:
                            newnode = Node(Board(curr.board.cars), curr.board.h3(5),0)
                        else:
                            newnode = Node(Board(curr.board.cars), curr.board.h3(5),0)

                        newnode.parent = curr                 
                        if(newnode.board.winConditionMet()):
                            sys.stdout = f
                            statessearched+=1



                            print("Runtime: %s seconds" % (time.time() - start_time))
                            self.findsteps(curr)
                            print("Number of moves: ", self.steps)
                            print("Search path length: %s states" % statessearched)
                            for node in self.path:
                                for path in node:
                                    print(""+path+", ", end ='')
                            print()

                            print("------")
                            print("---------------------------solution:---------------------------")
                            self.recursivePrinting(curr)
                            print("end")
                            sys.stdout = original_stdout
                            return

                                    #Apply the move
                        newnode.board.moveCar(c.carLetter, paths[i])
                        newnode.path.append(c.carLetter+": move:"+str(paths[i]))
                       

                        for node in self.opened:
                            if newnode.gn+newnode.h>node.gn+node.h:
                              #  print(str(newnode.h)+">"+str(node.h))
                                continue

                        if newnode in self.opened or newnode in self.closed:
                            continue

                        #Insert into open list (in sorted order)
                        self.printnode(newnode)
                        statessearched+=1
                        newnode.gn+=1
                        bisect.insort(self.opened, newnode)
                        
currentBoard = Board(boards[0].cars)
ucs = GBFS()
ucs.search(currentBoard,'GBFS_solution_h3_1.txt','HBFS_search_h3_1.txt')                        

currentBoard = Board(boards[1].cars)
ucs = GBFS()
ucs.search(currentBoard,'GBFS_solution_h3_2.txt','HBFS_search_h3_2.txt')
currentBoard = Board(boards[2].cars)
ucs = GBFS()
ucs.search(currentBoard,'GBFS_solution_h3_3.txt','HBFS_search_h3_3.txt')
currentBoard = Board(boards[3].cars)
ucs = GBFS()
ucs.search(currentBoard,'GBFS_solution_h3_4.txt','HBFS_search_h3_4.txt')
currentBoard = Board(boards[4].cars)
ucs = GBFS()
ucs.search(currentBoard,'GBFS_solution_h3_5.txt','HBFS_search_h3_5.txt')
currentBoard = Board(boards[5].cars)
ucs = GBFS()
ucs.search(currentBoard,'GBFS_solution_h3_6.txt','HBFS_search_h3_6.txt')

### Greedy Best First Search h4

In [9]:
import sys

    
class Node(object):
  def __init__(self, board, h1,gn):
    self.board = board
    self.h=0+h1
    self.gn=0+gn;
    self.path = []
    self.direction='none'
    self.parent = 0
  def __lt__(self, other):
        return self.h+self.gn < other.h+other.gn
  def __eq__(self, other):
    if isinstance(other,Node):
        return self.board == other.board
    
class GBFS(object):
  def __init__(self):
    self.opened = []
    self.closed = []
    self.steps=-1;
    self.path=[]
  def printnode(self,node):
    print(node.board)
    print("f(n):"+str(node.gn+node.h),end='')
    print(" g(n):"+str(node.gn),end='')
    print(" h(n):"+str(node.h),end=' ')
    
    for car in node.board.cars:
        if car.fuel<100:
            print(car.carLetter+": fuel:"+str(car.fuel), end = ' ')
            for i in range(len(car.path)):
                if car.direction==Direction.HORIZONTAL:
                    if(car.path[i]>0):
                        print("right:"+str(abs(car.path[i])), end = ' ')

                    if(car.path[i]<0):
                        print("left:"+str(abs(car.path[i])), end = ' ')


                if car.direction==Direction.VERTICAL:
                    if(car.path[i]>0):
                        print("Down:"+str(abs(car.path[i])), end = ' ')

                    if(car.path[i]<0):
                        print("UP:"+str(abs(car.path[i])), end = ' ')
               
    print("\n\n")
    
  def recursivePrinting(self, node):
    print(node.board)
    print("g(n):"+str(node.gn),end='')
    print(" h(n):"+str(node.h),end='')
    print(" f(n):"+str(node.gn+node.h))
    for car in node.board.cars:
        if car.fuel<100:
            print(car.carLetter+": fuel:"+str(car.fuel), end = ' ')
            for i in range(len(car.path)):
                if car.direction==Direction.HORIZONTAL:
                    if(car.path[i]>0):
                        print("right:"+str(abs(car.path[i])), end = ' ')

                    if(car.path[i]<0):
                        print("left:"+str(abs(car.path[i])), end = ' ')


                if car.direction==Direction.VERTICAL:
                    if(car.path[i]>0):
                        print("Down:"+str(abs(car.path[i])), end = ' ')

                    if(car.path[i]<0):
                        print("UP:"+str(abs(car.path[i])), end = ' ')
            print("")    
    print("\n\n")        
        
    
    if node.parent==0:
        return
    else:
      self.recursivePrinting(node.parent)
      
  def findsteps(self,node):
    self.steps+=1;
    self.path.append(node.path)
    if node.parent==0:
        return
    else:
      self.findsteps(node.parent)
    
  def findgn(self,node):
    
    if node.parent==0:
        return
    else:
      node.gn+=node.parent.h
      self.findgn(node.parent)
    
  def search(self, board,output_solution_file,output_search_file):
    original_stdout = sys.stdout # Save a reference to the original standard output

    with open(output_solution_file, 'w') as f, open(output_search_file, 'w') as fb:
        sys.stdout = f
        
        print("Initial board configuration")
        print()
        print(board)
        print("Car Fuel Available: ", end = '')
        for i in range(len(board.cars)):
            print(""+board.cars[i].carLetter+":"+str(board.cars[i].fuel)+", ", end ='')
        print()
        print("---------------------------running:---------------------------")
        #Initialise first Node with cost 0 (different for other algorithms)
        a = Node(copy.deepcopy(board), 0,0)
        self.opened.append(a)

        start_time = time.time()
        statessearched = 0
        sys.stdout = fb
        while(1):              
            #If open list is empty, exit
            
            if(len(self.opened)==0):
                
                print("No Solution")
                sys.stdout = f
                print("No Solution")
                sys.stdout = original_stdout
                return

            #Pop the best candidate node
            curr = self.opened.pop(0)

            #If the popped node is the solution, exit out and print stuff (later to file)



            #print("CHECKING CLOSED + OPEN: ")
            #print("-----")
            #for c in self.closed:
            #  print(c.board)
            #print("-----")

            #for c in self.opened:
            #  print(c.board)
            #print("----- END")

            #If it's not a solution insert into closed list and increment counter
            self.closed.append(curr)

          #
            #print()
            for c in curr.board.cars:

                paths = curr.board.findCarPaths(c)    
                if len(paths) > 0:  
                    #print("Car ", c.carLetter, " can move:", paths)

                    for i in range(len(paths)):
                        #Create a new node for successor
                        #Increase the cost of this new node (different in other algorithms)
                        newnode=0
                        if curr.parent!=0:
                            newnode = Node(Board(curr.board.cars), curr.board.h4(),0)
                        else:
                            newnode = Node(Board(curr.board.cars), curr.board.h4(),0)

                        newnode.parent = curr                 
                        if(newnode.board.winConditionMet()):
                            sys.stdout = f
                            statessearched+=1



                            print("Runtime: %s seconds" % (time.time() - start_time))
                            self.findsteps(curr)
                            print("Number of moves: ", self.steps)
                            print("Search path length: %s states" % statessearched)
                            for node in self.path:
                                for path in node:
                                    print(""+path+", ", end ='')
                            print()

                            print("------")
                            print("---------------------------solution:---------------------------")
                            self.recursivePrinting(curr)
                            print("end")
                            sys.stdout = original_stdout
                            return

                                    #Apply the move
                        newnode.board.moveCar(c.carLetter, paths[i])
                        newnode.path.append(c.carLetter+": move:"+str(paths[i]))
                       

                        for node in self.opened:
                            if newnode.gn+newnode.h>node.gn+node.h:
                              #  print(str(newnode.h)+">"+str(node.h))
                                continue

                        if newnode in self.opened or newnode in self.closed:
                            continue

                        #Insert into open list (in sorted order)
                        self.printnode(newnode)
                        statessearched+=1
                        newnode.gn+=1
                        bisect.insort(self.opened, newnode)
                        
currentBoard = Board(boards[0].cars)
ucs = GBFS()
ucs.search(currentBoard,'GBFS_solution_h4_1.txt','HBFS_search_h4_1.txt')                        

currentBoard = Board(boards[1].cars)
ucs = GBFS()
ucs.search(currentBoard,'GBFS_solution_h4_2.txt','HBFS_search_h4_2.txt')
currentBoard = Board(boards[2].cars)
ucs = GBFS()
ucs.search(currentBoard,'GBFS_solution_h4_3.txt','HBFS_search_h4_3.txt')
currentBoard = Board(boards[3].cars)
ucs = GBFS()
ucs.search(currentBoard,'GBFS_solution_h4_4.txt','HBFS_search_h4_4.txt')
currentBoard = Board(boards[4].cars)
ucs = GBFS()
ucs.search(currentBoard,'GBFS_solution_h4_5.txt','HBFS_search_h4_5.txt')
currentBoard = Board(boards[5].cars)
ucs = GBFS()
ucs.search(currentBoard,'GBFS_solution_h4_6.txt','HBFS_search_h4_6.txt')

### Algorithm A or A⋆(A/A⋆) h1

In [10]:
import sys

    
class Node(object):
  def __init__(self, board, h1,gn):
    self.board = board
    self.h=0+h1
    self.gn=0+gn;
    self.path = []
    self.direction='none'
    self.parent = 0
  def __lt__(self, other):
        return self.h+self.gn < other.h+other.gn
  def __eq__(self, other):
    if isinstance(other,Node):
        return self.board == other.board
    
class Astar(object):
  def __init__(self):
    self.opened = []
    self.closed = []
    self.steps=-1;
    self.path=[]
  def printnode(self,node):
    print(node.board)
    print("f(n):"+str(node.gn+node.h),end='')
    print(" g(n):"+str(node.gn),end='')
    print(" h(n):"+str(node.h),end=' ')
    
    for car in node.board.cars:
        if car.fuel<100:
            print(car.carLetter+": fuel:"+str(car.fuel), end = ' ')
            for i in range(len(car.path)):
                if car.direction==Direction.HORIZONTAL:
                    if(car.path[i]>0):
                        print("right:"+str(abs(car.path[i])), end = ' ')

                    if(car.path[i]<0):
                        print("left:"+str(abs(car.path[i])), end = ' ')


                if car.direction==Direction.VERTICAL:
                    if(car.path[i]>0):
                        print("Down:"+str(abs(car.path[i])), end = ' ')

                    if(car.path[i]<0):
                        print("UP:"+str(abs(car.path[i])), end = ' ')
               
    print("\n\n")
    
  def recursivePrinting(self, node):
    print(node.board)
    print("g(n):"+str(node.gn),end='')
    print(" h(n):"+str(node.h),end='')
    print(" f(n):"+str(node.gn+node.h))
    for car in node.board.cars:
        if car.fuel<100:
            print(car.carLetter+": fuel:"+str(car.fuel), end = ' ')
            for i in range(len(car.path)):
                if car.direction==Direction.HORIZONTAL:
                    if(car.path[i]>0):
                        print("right:"+str(abs(car.path[i])), end = ' ')

                    if(car.path[i]<0):
                        print("left:"+str(abs(car.path[i])), end = ' ')


                if car.direction==Direction.VERTICAL:
                    if(car.path[i]>0):
                        print("Down:"+str(abs(car.path[i])), end = ' ')

                    if(car.path[i]<0):
                        print("UP:"+str(abs(car.path[i])), end = ' ')
            print("")    
    print("\n\n")        
        
    
    if node.parent==0:
        return
    else:
      self.recursivePrinting(node.parent)
      
  def findsteps(self,node):
    self.steps+=1;
    self.path.append(node.path)
    if node.parent==0:
        return
    else:
      self.findsteps(node.parent)
    
  def findgn(self,node):
    
    if node.parent==0:
        return
    else:
      node.gn+=node.parent.h
      self.findgn(node.parent)
    
  def search(self, board,output_solution_file,output_search_file):
    original_stdout = sys.stdout # Save a reference to the original standard output

    with open(output_solution_file, 'w') as f, open(output_search_file, 'w') as fb:
        sys.stdout = f
        
        print("Initial board configuration")
        print()
        print(board)
        print("Car Fuel Available: ", end = '')
        for i in range(len(board.cars)):
            print(""+board.cars[i].carLetter+":"+str(board.cars[i].fuel)+", ", end ='')
        print()
        print("---------------------------running:---------------------------")
        #Initialise first Node with cost 0 (different for other algorithms)
        a = Node(copy.deepcopy(board), board.h1(),0)
        self.opened.append(a)

        start_time = time.time()
        statessearched = 0
        sys.stdout = fb
        while(1):              
            #If open list is empty, exit
            
            if(len(self.opened)==0):
                
                print("No Solution")
                sys.stdout = f
                print("No Solution")
                sys.stdout = original_stdout
                return

            #Pop the best candidate node
            curr = self.opened.pop(0)

            #If the popped node is the solution, exit out and print stuff (later to file)



            #print("CHECKING CLOSED + OPEN: ")
            #print("-----")
            #for c in self.closed:
            #  print(c.board)
            #print("-----")

            #for c in self.opened:
            #  print(c.board)
            #print("----- END")

            #If it's not a solution insert into closed list and increment counter
            self.closed.append(curr)

          #
            #print()
            for c in curr.board.cars:

                paths = curr.board.findCarPaths(c)    
                if len(paths) > 0:  
                    #print("Car ", c.carLetter, " can move:", paths)

                    for i in range(len(paths)):
                        #Create a new node for successor
                        #Increase the cost of this new node (different in other algorithms)
                        newnode=0
                        if curr.parent!=0:
                            newnode = Node(Board(curr.board.cars), curr.board.h1(),curr.parent.gn+1)
                        else:
                            newnode = Node(Board(curr.board.cars), curr.board.h1(),1)

                        newnode.parent = curr                 
                        if(newnode.board.winConditionMet()):
                            sys.stdout = f
                            statessearched+=1



                            print("Runtime: %s seconds" % (time.time() - start_time))
                            self.findsteps(curr)
                            print("Number of moves: ", self.steps)
                            print("Search path length: %s states" % statessearched)
                            for node in self.path:
                                for path in node:
                                    print(""+path+", ", end ='')
                            print()

                            print("------")
                            print("---------------------------solution:---------------------------")
                            self.recursivePrinting(curr)
                            print("end")
                            sys.stdout = original_stdout
                            return

                                    #Apply the move
                        newnode.board.moveCar(c.carLetter, paths[i])
                        newnode.path.append(c.carLetter+": move:"+str(paths[i]))
                       

                        for node in self.opened:
                            if newnode.gn+newnode.h>node.gn+node.h:
                              #  print(str(newnode.h)+">"+str(node.h))
                                continue

                        if newnode in self.opened or newnode in self.closed:
                            continue

                        #Insert into open list (in sorted order)
                        self.printnode(newnode)
                        statessearched+=1
                        newnode.gn+=1
                        bisect.insort(self.opened, newnode)

currentBoard = Board(boards[0].cars)
astar = Astar()
astar.search(currentBoard,'Astar_solution_h1_1.txt','Astar_search_h1_1.txt')
currentBoard = Board(boards[1].cars)
astar = Astar()
astar.search(currentBoard,'Astar_solution_h1_2.txt','Astar_search_h1_2.txt')
currentBoard = Board(boards[2].cars)
astar = Astar()
astar.search(currentBoard,'Astar_solution_h1_3.txt','Astar_search_h1_3.txt')
currentBoard = Board(boards[3].cars)
astar = Astar()
astar.search(currentBoard,'Astar_solution_h1_4.txt','Astar_search_h1_4.txt')
currentBoard = Board(boards[4].cars)
astar = Astar()
astar.search(currentBoard,'Astar_solution_h1_5.txt','Astar_search_h1_5.txt')
currentBoard = Board(boards[5].cars)
astar = Astar()
astar.search(currentBoard,'Astar_solution_h1_6.txt','Astar_search_h1_6.txt')

### Algorithm A or A⋆(A/A⋆) h2

In [11]:
import sys

    
class Node(object):
  def __init__(self, board, h1,gn):
    self.board = board
    self.h=0+h1
    self.gn=0+gn;
    self.path = []
    self.direction='none'
    self.parent = 0
  def __lt__(self, other):
        return self.h+self.gn < other.h+other.gn
  def __eq__(self, other):
    if isinstance(other,Node):
        return self.board == other.board
    
class Astar(object):
  def __init__(self):
    self.opened = []
    self.closed = []
    self.steps=-1;
    self.path=[]
  def printnode(self,node):
    print(node.board)
    print("f(n):"+str(node.gn+node.h),end='')
    print(" g(n):"+str(node.gn),end='')
    print(" h(n):"+str(node.h),end=' ')
    
    for car in node.board.cars:
        if car.fuel<100:
            print(car.carLetter+": fuel:"+str(car.fuel), end = ' ')
            for i in range(len(car.path)):
                if car.direction==Direction.HORIZONTAL:
                    if(car.path[i]>0):
                        print("right:"+str(abs(car.path[i])), end = ' ')

                    if(car.path[i]<0):
                        print("left:"+str(abs(car.path[i])), end = ' ')


                if car.direction==Direction.VERTICAL:
                    if(car.path[i]>0):
                        print("Down:"+str(abs(car.path[i])), end = ' ')

                    if(car.path[i]<0):
                        print("UP:"+str(abs(car.path[i])), end = ' ')
               
    print("\n\n")
    
  def recursivePrinting(self, node):
    print(node.board)
    print("g(n):"+str(node.gn),end='')
    print(" h(n):"+str(node.h),end='')
    print(" f(n):"+str(node.gn+node.h))
    for car in node.board.cars:
        if car.fuel<100:
            print(car.carLetter+": fuel:"+str(car.fuel), end = ' ')
            for i in range(len(car.path)):
                if car.direction==Direction.HORIZONTAL:
                    if(car.path[i]>0):
                        print("right:"+str(abs(car.path[i])), end = ' ')

                    if(car.path[i]<0):
                        print("left:"+str(abs(car.path[i])), end = ' ')


                if car.direction==Direction.VERTICAL:
                    if(car.path[i]>0):
                        print("Down:"+str(abs(car.path[i])), end = ' ')

                    if(car.path[i]<0):
                        print("UP:"+str(abs(car.path[i])), end = ' ')
            print("")    
    print("\n\n")        
        
    
    if node.parent==0:
        return
    else:
      self.recursivePrinting(node.parent)
      
  def findsteps(self,node):
    self.steps+=1;
    self.path.append(node.path)
    if node.parent==0:
        return
    else:
      self.findsteps(node.parent)
    
  def findgn(self,node):
    
    if node.parent==0:
        return
    else:
      node.gn+=node.parent.h
      self.findgn(node.parent)
    
  def search(self, board,output_solution_file,output_search_file):
    original_stdout = sys.stdout # Save a reference to the original standard output

    with open(output_solution_file, 'w') as f, open(output_search_file, 'w') as fb:
        sys.stdout = f
        
        print("Initial board configuration")
        print()
        print(board)
        print("Car Fuel Available: ", end = '')
        for i in range(len(board.cars)):
            print(""+board.cars[i].carLetter+":"+str(board.cars[i].fuel)+", ", end ='')
        print()
        print("---------------------------running:---------------------------")
        #Initialise first Node with cost 0 (different for other algorithms)
        a = Node(copy.deepcopy(board), board.h2(),0)
        self.opened.append(a)

        start_time = time.time()
        statessearched = 0
        sys.stdout = fb
        while(1):              
            #If open list is empty, exit
            
            if(len(self.opened)==0):
                
                print("No Solution")
                sys.stdout = f
                print("No Solution")
                sys.stdout = original_stdout
                return

            #Pop the best candidate node
            curr = self.opened.pop(0)

            #If the popped node is the solution, exit out and print stuff (later to file)



            #print("CHECKING CLOSED + OPEN: ")
            #print("-----")
            #for c in self.closed:
            #  print(c.board)
            #print("-----")

            #for c in self.opened:
            #  print(c.board)
            #print("----- END")

            #If it's not a solution insert into closed list and increment counter
            self.closed.append(curr)

          #
            #print()
            for c in curr.board.cars:

                paths = curr.board.findCarPaths(c)    
                if len(paths) > 0:  
                    #print("Car ", c.carLetter, " can move:", paths)

                    for i in range(len(paths)):
                        #Create a new node for successor
                        #Increase the cost of this new node (different in other algorithms)
                        newnode=0
                        if curr.parent!=0:
                            newnode = Node(Board(curr.board.cars), curr.board.h2(),curr.parent.gn+1)
                        else:
                            newnode = Node(Board(curr.board.cars), curr.board.h2(),1)

                        newnode.parent = curr                 
                        if(newnode.board.winConditionMet()):
                            sys.stdout = f
                            statessearched+=1



                            print("Runtime: %s seconds" % (time.time() - start_time))
                            self.findsteps(curr)
                            print("Number of moves: ", self.steps)
                            print("Search path length: %s states" % statessearched)
                            for node in self.path:
                                for path in node:
                                    print(""+path+", ", end ='')
                            print()

                            print("------")
                            print("---------------------------solution:---------------------------")
                            self.recursivePrinting(curr)
                            print("end")
                            sys.stdout = original_stdout
                            return

                                    #Apply the move
                        newnode.board.moveCar(c.carLetter, paths[i])
                        newnode.path.append(c.carLetter+": move:"+str(paths[i]))
                       

                        for node in self.opened:
                            if newnode.gn+newnode.h>node.gn+node.h:
                              #  print(str(newnode.h)+">"+str(node.h))
                                continue

                        if newnode in self.opened or newnode in self.closed:
                            continue

                        #Insert into open list (in sorted order)
                        self.printnode(newnode)
                        statessearched+=1
                        newnode.gn+=1
                        bisect.insort(self.opened, newnode)

currentBoard = Board(boards[0].cars)
astar = Astar()
astar.search(currentBoard,'Astar_solution_h2_1.txt','Astar_search_h2_1.txt')
currentBoard = Board(boards[1].cars)
astar = Astar()
astar.search(currentBoard,'Astar_solution_h2_2.txt','Astar_search_h2_2.txt')
currentBoard = Board(boards[2].cars)
astar = Astar()
astar.search(currentBoard,'Astar_solution_h2_3.txt','Astar_search_h2_3.txt')
currentBoard = Board(boards[3].cars)
astar = Astar()
astar.search(currentBoard,'Astar_solution_h2_4.txt','Astar_search_h2_4.txt')
currentBoard = Board(boards[4].cars)
astar = Astar()
astar.search(currentBoard,'Astar_solution_h2_5.txt','Astar_search_h2_5.txt')
currentBoard = Board(boards[5].cars)
astar = Astar()
astar.search(currentBoard,'Astar_solution_h2_6.txt','Astar_search_h2_6.txt')

### Algorithm A or A⋆(A/A⋆) h3

In [12]:
class Node(object):
  def __init__(self, board, h1,gn):
    self.board = board
    self.h=0+h1
    self.gn=0+gn;
    self.path = []
    self.direction='none'
    self.parent = 0
  def __lt__(self, other):
        return self.h+self.gn < other.h+other.gn
  def __eq__(self, other):
    if isinstance(other,Node):
        return self.board == other.board
    
class Astar(object):
  def __init__(self):
    self.opened = []
    self.closed = []
    self.steps=-1;
    self.path=[]
  def printnode(self,node):
    print(node.board)
    print("f(n):"+str(node.gn+node.h),end='')
    print(" g(n):"+str(node.gn),end='')
    print(" h(n):"+str(node.h),end=' ')
    
    for car in node.board.cars:
        if car.fuel<100:
            print(car.carLetter+": fuel:"+str(car.fuel), end = ' ')
            for i in range(len(car.path)):
                if car.direction==Direction.HORIZONTAL:
                    if(car.path[i]>0):
                        print("right:"+str(abs(car.path[i])), end = ' ')

                    if(car.path[i]<0):
                        print("left:"+str(abs(car.path[i])), end = ' ')


                if car.direction==Direction.VERTICAL:
                    if(car.path[i]>0):
                        print("Down:"+str(abs(car.path[i])), end = ' ')

                    if(car.path[i]<0):
                        print("UP:"+str(abs(car.path[i])), end = ' ')
               
    print("\n\n")
    
  def recursivePrinting(self, node):
    print(node.board)
    print("g(n):"+str(node.gn),end='')
    print(" h(n):"+str(node.h),end='')
    print(" f(n):"+str(node.gn+node.h))
    for car in node.board.cars:
        if car.fuel<100:
            print(car.carLetter+": fuel:"+str(car.fuel), end = ' ')
            for i in range(len(car.path)):
                if car.direction==Direction.HORIZONTAL:
                    if(car.path[i]>0):
                        print("right:"+str(abs(car.path[i])), end = ' ')

                    if(car.path[i]<0):
                        print("left:"+str(abs(car.path[i])), end = ' ')


                if car.direction==Direction.VERTICAL:
                    if(car.path[i]>0):
                        print("Down:"+str(abs(car.path[i])), end = ' ')

                    if(car.path[i]<0):
                        print("UP:"+str(abs(car.path[i])), end = ' ')
            print("")    
    print("\n\n")        
        
    
    if node.parent==0:
        return
    else:
      self.recursivePrinting(node.parent)
      
  def findsteps(self,node):
    self.steps+=1;
    self.path.append(node.path)
    if node.parent==0:
        return
    else:
      self.findsteps(node.parent)
    
  def findgn(self,node):
    
    if node.parent==0:
        return
    else:
      node.gn+=node.parent.h
      self.findgn(node.parent)
    
  def search(self, board,output_solution_file,output_search_file):
    original_stdout = sys.stdout # Save a reference to the original standard output

    with open(output_solution_file, 'w') as f, open(output_search_file, 'w') as fb:
        sys.stdout = f
        
        print("Initial board configuration")
        print()
        print(board)
        print("Car Fuel Available: ", end = '')
        for i in range(len(board.cars)):
            print(""+board.cars[i].carLetter+":"+str(board.cars[i].fuel)+", ", end ='')
        print()
        print("---------------------------running:---------------------------")
        #Initialise first Node with cost 0 (different for other algorithms)
        a = Node(copy.deepcopy(board), board.h3(5),0)
        self.opened.append(a)

        start_time = time.time()
        statessearched = 0
        sys.stdout = fb
        while(1):              
            #If open list is empty, exit
            
            if(len(self.opened)==0):
                
                print("No Solution")
                sys.stdout = f
                print("No Solution")
                sys.stdout = original_stdout
                return

            #Pop the best candidate node
            curr = self.opened.pop(0)

            #If the popped node is the solution, exit out and print stuff (later to file)



            #print("CHECKING CLOSED + OPEN: ")
            #print("-----")
            #for c in self.closed:
            #  print(c.board)
            #print("-----")

            #for c in self.opened:
            #  print(c.board)
            #print("----- END")

            #If it's not a solution insert into closed list and increment counter
            self.closed.append(curr)

          #
            #print()
            for c in curr.board.cars:

                paths = curr.board.findCarPaths(c)    
                if len(paths) > 0:  
                    #print("Car ", c.carLetter, " can move:", paths)

                    for i in range(len(paths)):
                        #Create a new node for successor
                        #Increase the cost of this new node (different in other algorithms)
                        newnode=0
                        if curr.parent!=0:
                            newnode = Node(Board(curr.board.cars), curr.board.h3(5),curr.parent.gn+1)
                        else:
                            newnode = Node(Board(curr.board.cars), curr.board.h3(5),1)

                        newnode.parent = curr                 
                        if(newnode.board.winConditionMet()):
                            sys.stdout = f
                            statessearched+=1



                            print("Runtime: %s seconds" % (time.time() - start_time))
                            self.findsteps(curr)
                            print("Number of moves: ", self.steps)
                            print("Search path length: %s states" % statessearched)
                            for node in self.path:
                                for path in node:
                                    print(""+path+", ", end ='')
                            print()

                            print("------")
                            print("---------------------------solution:---------------------------")
                            self.recursivePrinting(curr)
                            print("end")
                            sys.stdout = original_stdout
                            return

                                    #Apply the move
                        newnode.board.moveCar(c.carLetter, paths[i])
                        newnode.path.append(c.carLetter+": move:"+str(paths[i]))
                       

                        for node in self.opened:
                            if newnode.gn+newnode.h>node.gn+node.h:
                              #  print(str(newnode.h)+">"+str(node.h))
                                continue

                        if newnode in self.opened or newnode in self.closed:
                            continue

                        #Insert into open list (in sorted order)
                        self.printnode(newnode)
                        statessearched+=1
                        newnode.gn+=1
                        bisect.insort(self.opened, newnode)

currentBoard = Board(boards[0].cars)
astar = Astar()
astar.search(currentBoard,'Astar_solution_h3_1.txt','Astar_search_h3_1.txt')
currentBoard = Board(boards[1].cars)
astar = Astar()
astar.search(currentBoard,'Astar_solution_h3_2.txt','Astar_search_h3_2.txt')
currentBoard = Board(boards[2].cars)
astar = Astar()
astar.search(currentBoard,'Astar_solution_h3_3.txt','Astar_search_h3_3.txt')
currentBoard = Board(boards[3].cars)
astar = Astar()
astar.search(currentBoard,'Astar_solution_h3_4.txt','Astar_search_h3_4.txt')
currentBoard = Board(boards[4].cars)
astar = Astar()
astar.search(currentBoard,'Astar_solution_h3_5.txt','Astar_search_h3_5.txt')
currentBoard = Board(boards[5].cars)
astar = Astar()
astar.search(currentBoard,'Astar_solution_h3_6.txt','Astar_search_h3_6.txt')

### Algorithm A or A⋆(A/A⋆) h4

In [ ]:
class Node(object):
  def __init__(self, board, h1,gn):
    self.board = board
    self.h=0+h1
    self.gn=0+gn;
    self.path = []
    self.direction='none'
    self.parent = 0
  def __lt__(self, other):
        return self.h+self.gn < other.h+other.gn
  def __eq__(self, other):
    if isinstance(other,Node):
        return self.board == other.board
    
class Astar(object):
  def __init__(self):
    self.opened = []
    self.closed = []
    self.steps=-1;
    self.path=[]
  def printnode(self,node):
    print(node.board)
    print("f(n):"+str(node.gn+node.h),end='')
    print(" g(n):"+str(node.gn),end='')
    print(" h(n):"+str(node.h),end=' ')
    
    for car in node.board.cars:
        if car.fuel<100:
            print(car.carLetter+": fuel:"+str(car.fuel), end = ' ')
            for i in range(len(car.path)):
                if car.direction==Direction.HORIZONTAL:
                    if(car.path[i]>0):
                        print("right:"+str(abs(car.path[i])), end = ' ')

                    if(car.path[i]<0):
                        print("left:"+str(abs(car.path[i])), end = ' ')


                if car.direction==Direction.VERTICAL:
                    if(car.path[i]>0):
                        print("Down:"+str(abs(car.path[i])), end = ' ')

                    if(car.path[i]<0):
                        print("UP:"+str(abs(car.path[i])), end = ' ')
               
    print("\n\n")
    
  def recursivePrinting(self, node):
    print(node.board)
    print("g(n):"+str(node.gn),end='')
    print(" h(n):"+str(node.h),end='')
    print(" f(n):"+str(node.gn+node.h))
    for car in node.board.cars:
        if car.fuel<100:
            print(car.carLetter+": fuel:"+str(car.fuel), end = ' ')
            for i in range(len(car.path)):
                if car.direction==Direction.HORIZONTAL:
                    if(car.path[i]>0):
                        print("right:"+str(abs(car.path[i])), end = ' ')

                    if(car.path[i]<0):
                        print("left:"+str(abs(car.path[i])), end = ' ')


                if car.direction==Direction.VERTICAL:
                    if(car.path[i]>0):
                        print("Down:"+str(abs(car.path[i])), end = ' ')

                    if(car.path[i]<0):
                        print("UP:"+str(abs(car.path[i])), end = ' ')
            print("")    
    print("\n\n")        
        
    
    if node.parent==0:
        return
    else:
      self.recursivePrinting(node.parent)
      
  def findsteps(self,node):
    self.steps+=1;
    self.path.append(node.path)
    if node.parent==0:
        return
    else:
      self.findsteps(node.parent)
    
  def findgn(self,node):
    
    if node.parent==0:
        return
    else:
      node.gn+=node.parent.h
      self.findgn(node.parent)
    
  def search(self, board,output_solution_file,output_search_file):
    original_stdout = sys.stdout # Save a reference to the original standard output

    with open(output_solution_file, 'w') as f, open(output_search_file, 'w') as fb:
        sys.stdout = f
        
        print("Initial board configuration")
        print()
        print(board)
        print("Car Fuel Available: ", end = '')
        for i in range(len(board.cars)):
            print(""+board.cars[i].carLetter+":"+str(board.cars[i].fuel)+", ", end ='')
        print()
        print("---------------------------running:---------------------------")
        #Initialise first Node with cost 0 (different for other algorithms)
        a = Node(copy.deepcopy(board), board.h4(),0)
        self.opened.append(a)

        start_time = time.time()
        statessearched = 0
        sys.stdout = fb
        while(1):              
            #If open list is empty, exit
            
            if(len(self.opened)==0):
                
                print("No Solution")
                sys.stdout = f
                print("No Solution")
                sys.stdout = original_stdout
                return

            #Pop the best candidate node
            curr = self.opened.pop(0)

            #If the popped node is the solution, exit out and print stuff (later to file)



            #print("CHECKING CLOSED + OPEN: ")
            #print("-----")
            #for c in self.closed:
            #  print(c.board)
            #print("-----")

            #for c in self.opened:
            #  print(c.board)
            #print("----- END")

            #If it's not a solution insert into closed list and increment counter
            self.closed.append(curr)

          #
            #print()
            for c in curr.board.cars:

                paths = curr.board.findCarPaths(c)    
                if len(paths) > 0:  
                    #print("Car ", c.carLetter, " can move:", paths)

                    for i in range(len(paths)):
                        #Create a new node for successor
                        #Increase the cost of this new node (different in other algorithms)
                        newnode=0
                        if curr.parent!=0:
                            newnode = Node(Board(curr.board.cars), curr.board.h4(),curr.parent.gn+1)
                        else:
                            newnode = Node(Board(curr.board.cars), curr.board.h4(),1)

                        newnode.parent = curr                 
                        if(newnode.board.winConditionMet()):
                            sys.stdout = f
                            statessearched+=1



                            print("Runtime: %s seconds" % (time.time() - start_time))
                            self.findsteps(curr)
                            print("Number of moves: ", self.steps)
                            print("Search path length: %s states" % statessearched)
                            for node in self.path:
                                for path in node:
                                    print(""+path+", ", end ='')
                            print()

                            print("------")
                            print("---------------------------solution:---------------------------")
                            self.recursivePrinting(curr)
                            print("end")
                            sys.stdout = original_stdout
                            return

                                    #Apply the move
                        newnode.board.moveCar(c.carLetter, paths[i])
                        newnode.path.append(c.carLetter+": move:"+str(paths[i]))
                       

                        for node in self.opened:
                            if newnode.gn+newnode.h>node.gn+node.h:
                              #  print(str(newnode.h)+">"+str(node.h))
                                continue

                        if newnode in self.opened or newnode in self.closed:
                            continue

                        #Insert into open list (in sorted order)
                        self.printnode(newnode)
                        statessearched+=1
                        newnode.gn+=1
                        bisect.insort(self.opened, newnode)

currentBoard = Board(boards[0].cars)
astar = Astar()
astar.search(currentBoard,'Astar_solution_h4_1.txt','Astar_search_h4_1.txt')
currentBoard = Board(boards[1].cars)
astar = Astar()
astar.search(currentBoard,'Astar_solution_h4_2.txt','Astar_search_h4_2.txt')
currentBoard = Board(boards[2].cars)
astar = Astar()
astar.search(currentBoard,'Astar_solution_h4_3.txt','Astar_search_h4_3.txt')
currentBoard = Board(boards[3].cars)
astar = Astar()
astar.search(currentBoard,'Astar_solution_h4_4.txt','Astar_search_h4_4.txt')
currentBoard = Board(boards[4].cars)
astar = Astar()
astar.search(currentBoard,'Astar_solution_h4_5.txt','Astar_search_h4_5.txt')
currentBoard = Board(boards[5].cars)
astar = Astar()
astar.search(currentBoard,'Astar_solution_h4_6.txt','Astar_search_h4_6.txt')